In [1]:
import re
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

In [2]:
topics_params = {    
                213960: 303, 214534: 93, 218669: 1006,
                218775: 999, 220708: 996, 220716: 992, 
                217612: 896, 216127: 871, 218207: 870,
                220172: 869, 216526: 771, 215199: 771, 
                220574: 749, 210668: 742, 209857: 739,
                217336: 734, 218957: 697, 156892: 369,
                215962: 676, 212651: 630, 211573: 763,
                218458: 2322, 210667: 669, 217393: 1040, 
                218082: 1009, 210495: 534, 214915: 530,
                204354: 529, 214493: 523, 
                222196: 2, 216840: 181, 222005: 126,
                215736: 130, 215184: 144, 215967: 419, 214282: 369,
                213224: 344, 215305: 490, 212248: 424, 215124: 262, 211629: 925,
                210753: 754, 214918: 729, 215491: 729, 188351: 676, 213296: 666,
                181554: 696, 219717: 662,  166276: 423,  218047: 1270, 218707: 1588
}

In [3]:
def find_soup(i, topic):
    try:
        URL = f"https://prodota.ru/forum/topic/{topic}/page/{i}/"
        page = requests.get(URL, verify=False)
        soup = BeautifulSoup(page.content, "html.parser")
        
    except requests.exceptions.ConnectionError:
        requests.status_code = "Connection refused"
    
    return soup


def find_replics(elements):
    reps1 = []
    reps2 = []
    for x in elements:
        with_cit = x.find("div", {"class": "ipsQuote_contents"})
        if with_cit:
            
            trash_classes = with_cit.find('blockquote',class_='ipsQuote')

            if trash_classes is not None:
                trash_classes.decompose()
            
            replica_1_list = with_cit.find_all('p')

            replica_1 = ' '.join([el.get_text() for el in replica_1_list])

        
            cur_text = x.find("div", {"class": "ipsType_normal ipsType_richText ipsContained"})
            if cur_text:

                quote_class_comment = cur_text.find('blockquote',class_='ipsQuote')
                if quote_class_comment is not None:
                     quote_class_comment.decompose()

                replica_2_list = cur_text.find_all('p')
                replica_2 = ' '.join([el.get_text() for el in replica_2_list])

                if replica_2 and replica_1:
                    reps1.append(replica_1)
                    reps2.append(replica_2) 
    

    return reps1, reps2

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
for k, v in tqdm(topics_params.items()):
    data = pd.DataFrame(columns = ['Реплика 1', 'Реплика 2'])
    for i in tqdm(range(v)):
        try:
            soup = find_soup(i, k)
            elements = soup.find_all("div", {"class": "cPost_contentWrap ipsPad"})  
            replica_1, replica_2 = find_replics(elements)
            for replics1, replics2 in zip(replica_1, replica_2):

                data = data.append({'Реплика 1': replics1, 'Реплика 2': replics2},  ignore_index=True)
        except:
            pass
    
    print(data)

    data.to_csv(f'dota{k}.csv', index=False)



                                               Реплика 1  \
0      \n\t\t\tна первой еб патч гавно спасибо \n\t\t...   
1      \n \n \n\n \n\t\t\tЭто должно быть на первой\n...   
2       \n\t\t\t \n\t\t \n\t\t\tпервый и на первой\n\t\t   
3      \n\t\t\tна первой еб патч гавно спасибо \n\t\t...   
4      \n \n \n\n \n\t\t\tЭто должно быть на первой\n...   
...                                                  ...   
10058                                               \n\n   
10059  \n\t\t\tну объективно мк параша в мид\n\t\t \n...   
10060            \n\t\t\tоткуда деньги рассказывай\n\t\t   
10061  \n\n\t\t\tabsolute in doubt🙄😵\n \n\t\t\t \n\t\...   
10062                    \n@Ember_Xinit че катанём\n\t\t   

                                               Реплика 2  
0        \n\tЯ ТВОЮ МАТЬ НАХУЙ СОЖРУ ДОЛБОЕБ НИКЧЕМНЫЙ\n  
1      \n\tРЕБЯТ ВЫ ДИБИЛЫ? ВАНЬКА СКАЗАЛ ОНЛИ РАЗГОВ...  
2      \n\tтак это не тема про стримы и все этого \n ...  
3        \n\tЯ ТВОЮ МАТЬ НАХУЙ СОЖРУ ДОЛБО


                                             Реплика 1  \
0    \npOJARNEK.Ставлю два отарика в доте из двух.\...   
1    \npOJARNEK.Ставлю два отарика в доте из двух.\...   
2    \npOJARNEK.Ставлю два отарика в доте из двух.\...   
3    \npOJARNEK.Ставлю два отарика в доте из двух.\...   
4    \npOJARNEK.Ставлю два отарика в доте из двух.\...   
..                                                 ...   
510  \n\t\t\tНе играл в прошлый лабиринт аганима, е...   
511  \n\t\t\tПривет сегодня на доверии приобрел меч...   
512  \n\t\t\tПривет сегодня на доверии приобрел меч...   
513         \n\t\t\tпроценты гаранту не забываем\n\t\t   
514  \n\t\t\tПродам аркейн на Джагера. 1500 рубчинс...   

                                             Реплика 2  
0         Продам вот такую хуевину предпочтительно ...  
1                                            трейдебл   
2    меч на свена за 0.65 отдать готов если кому ин...  
3    есть тут скупщики 0.7?  тот акк с мечом на све...  
4    Продаётся ак


                                              Реплика 1  \
0     \n\t\t\t\tя думал на земле нет таких людей кот...   
1     \n\t\t\t\tя думал на земле нет таких людей кот...   
2     \n  у меня турбачи ищутся минут 20-30 как эту ...   
3     \n\t\t\t\tя думал на земле нет таких людей кот...   
4     \n\t\t\tмамуууууууууууууууууу ебаллллллллллллл...   
...                                                 ...   
2834  \n\t\t\tфармат о чем с девочками говорить посо...   
2835  \n\t\t\tПродам акк 6170\n\t\t \n\t\t\tбихевойр...   
2836  \n\t\t\tа ты гарантию даешь что их не спиздят?...   
2837                       \n\t\t\tЯ черная мамба\n\t\t   
2838  \n\t\t\tмне покусала яица твоя никчемная мама\...   

                                              Реплика 2  
0     \n\tмамуууууууууууууууууу ебаллллллллллллллллл...  
1     \n\tмамуууууууууууууууууу ебаллллллллллллллллл...  
2     \n\tНу если ещё работает, то посмотри нескольк...  
3                                                    \n  



                                              Реплика 1  \
0     \n\t\t\t \n\t\t \n\n\t\t\tИгроки, ранее выступ...   
1     \n\t\t\tПерспективно. Но их постигнет судьба В...   
2          \n\t\t\tНа первой у будущих победителей инта   
3     \n\t\t\tВПполар стали основным составом ВП с м...   
4     \n\t\t\t \n\t\t \n\n\t\t\tИгроки, ранее выступ...   
...                                                 ...   
7761  \n\t\t\tБудь ты здравомыслящим человеком, ты б...   
7762  \n\t\t\tВаня не был первым претендентом на рол...   
7763  \n\t\t\tОчень мощнo, так кто был их первым выб...   
7764              \n\t\t\tграц нави с топ-2 снг-помойки   
7765  \n\t\t\tиграл щас в пабе с челом, он сказал, ч...   

                                              Реплика 2  
0     \n \n \n«Молодой Одаренный Человек»\n \n\t \n ...  
1     \n\tВПполар стали основным составом ВП с миним...  
2                                               \n\t \n  
3     \n\tПриятно познакомиться дебил, но я вообще т...  
